In [112]:
import re
issn = dict()
def get_issn(journal):
    global issn
    pattern = re.compile(r'(\d{4}-\d{4})', re.IGNORECASE)
    
    if journal in issn: return issn[journal]
    page = "https://dblp.org/db/journals/"+journal+"/index.html";
    webpage = requests.get(page)
    soup = BeautifulSoup(webpage.content, 'html.parser')
    issn_text = soup.findAll(text=re.compile(r'\d{4}-\d{4}'))
    if len(issn_text)==0: 
        issn[journal] = ""
        return ""
    print(journal)
    issn_text = pattern.findall(str(issn_text[0]))
    issn[journal] = issn_text
    return issn[journal]


get_issn("jgo")
    

jgo


['0925-5001', '1573-2916']

In [105]:
import sys
import time
import requests
from bs4 import BeautifulSoup

#todas las listas (carga inicial)
papers_dict = {}
lists=[["iaraya","https://dblp.org/pid/88/3432.html"],["rsoto","https://dblp.org/pid/41/5788.html"],
       ["bcrawford","https://dblp.org/pid/99/4622.html"],["hallende","https://dblp.org/pid/10/649.html"],
      ["wpalma","https://dblp.org/pid/02/5664.html"],["gcabrera","https://dblp.org/pid/146/1064.html"],
      ["crusu","https://dblp.org/pid/38/4610.html"],["ccubillos","https://dblp.org/pid/68/4753.html"],
      ["dquinones","https://dblp.org/pid/164/6412.html"],["rvillarroel","https://dblp.org/pid/33/2661.html"],
       ["lperez","https://dblp.org/pid/26/11291.html"],["scano","https://dblp.org/pid/166/1026.html"]]
#lists=[["iaraya","https://dblp.org/pid/88/3432.html"]]

for l in lists:  
    mlist = []
    webpage = requests.get(l[1])
    soup = BeautifulSoup(webpage.content, 'html.parser')
    for nr in soup.find_all('div', class_="nr"):
        data = nr.find_next('cite', class_="data")
        title=data.find_next('span',class_="title")
        journal =title.find_next_sibling('a')
        if journal==None: continue
        mlist.append( (journal['href'].split('/')[5], data.text) ) 
    papers_dict[l[0]] = mlist 

In [114]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd 

file = r'JCR-2019.xls'
quartil = dict()
for sheet in range(4,0,-1):
    P = pd.read_excel(file, sheet_name=sheet,header=3)["ISSN"].tolist()
    for p in P:
        quartil[p]=sheet

In [117]:
import re
year="2021"
for p in papers_dict:
    print(p)
    for pp in range(len(papers_dict[p])):
        #print(papers_dict[p][pp])
        res = re.search(r"("+year+")", papers_dict[p][pp][1])
        if res: 
            for issn_ in get_issn(papers_dict[p][pp][0]):
                if issn_ in quartil:
                    print(papers_dict[p][pp][1] + ", Q" + str(quartil[issn_]))
                    break
            #print(papers_dict[p][pp][1])

iaraya
Víctor Reyes, Ignacio Araya: AbsTaylor: upper bounding with inner regions in nonlinear continuous global optimization problems. J. Glob. Optim. 79(2): 413-429 (2021), Q1
Ignacio Araya, Damir Aliquintui, Franco Ardiles, Braulio Lobo: Nonlinear biobjective optimization: improving the upper envelope using feasible line segments. J. Glob. Optim. 79(2): 503-520 (2021), Q1
rsoto
José Lemus-Romani, Borja Alonso, Jose Luis Moura, Broderick Crawford, Ricardo Soto, Felipe González: Limited Stop Services Design Considering Variable Dwell Time and Operating Capacity Constraints. IEEE Access 9: 30359-30373 (2021), Q1
bcrawford
José Lemus-Romani, Borja Alonso, Jose Luis Moura, Broderick Crawford, Ricardo Soto, Felipe González: Limited Stop Services Design Considering Variable Dwell Time and Operating Capacity Constraints. IEEE Access 9: 30359-30373 (2021), Q1
hallende
Gabriel Villavicencio Arancibia, Osvaldo Piña Bustamante, Gabriel Hermosilla Vigneau, Héctor Allende-Cid, Gonzalo Suazo Fuente

In [82]:
papers_dict[p][pp][0]

'hm'